In [1]:
import sys
import os
import json
import pickle
import pandas as pd
import fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import re
import cv2
import PIL
import math
from pdf2image import convert_from_path, convert_from_bytes

In [2]:
def get_file_by_dict(json_output):
    '''
    Given json output from OCR, construct a dictionary to better represent the data
    '''
    file_as_dict = {'words':[], 'ymin_ymax':[], 'page_id':[], 'page_dimensions':[], 'line_merge_next':[], 'bboxes': []}
    for page in json_output['pages']:
        for block in page['blocks']:
            for idx, line in enumerate(block['lines']):
                ((xmin, ymin),(xmax, ymax)) = line['geometry']

                
                file_as_dict['words'].append([line['words'][i]['value'] for i in range(len(line['words']))])
                file_as_dict['bboxes'].append([line['words'][ii]['geometry'] for ii in range(len(line['words']))])
                
                file_as_dict['ymin_ymax'].append((ymin, ymax))
                file_as_dict['page_id'].append(page['page_idx'])
                file_as_dict['page_dimensions'].append(page['dimensions'])

    return file_as_dict

def getIOU(segment1, segment2, threshold):
    '''
    Merging algorithm for line geometries. If lines are in different blocks but have similar line coordinates, we will be 
    able to "merge" them as one line this way. Function needed because OCR model does not always treat the same line
    as an item within the same block
    '''
    ymin1, ymax1 = segment1
    ymin2, ymax2 = segment2
    
    less_ymax = min(ymax1, ymax2)
    less_ymin = min(ymin1, ymin2)
    greater_ymax = max(ymax1, ymax2)
    greater_ymin = max(ymin1, ymin2)
    
    intersection = less_ymax - greater_ymin
    
    # no overlap
    if intersection < 0:
        return False
    
    union = greater_ymax - less_ymin
    
    if intersection / union > threshold:
        return True
    return False

def get_lines_to_merge(file_as_dict, threshold):
    lines_to_merge_indices = []
    for i in range(0, len(file_as_dict['words'])):
        for j in range(i+1, len(file_as_dict['words'])):
            if (file_as_dict['page_id'][i] == file_as_dict['page_id'][j]) and getIOU(file_as_dict['ymin_ymax'][i], 
                                                                                     file_as_dict['ymin_ymax'][j], 
                                                                                     threshold):
                lines_to_merge_indices.append((i,j))
    return lines_to_merge_indices

def line_merging(lines_to_merge_indices):
    follows = dict()
    lines = dict()
    for i, j in lines_to_merge_indices:

        if i not in lines:
            if i not in follows:
                lines[i] = [i, j]
                follows[j] = [i]
            else:  
                one_link_back = follows[i][0]
                while one_link_back in follows:
                    one_link_back = follows[one_link_back][0]
                if j not in lines[one_link_back]:
                    lines[follows[i][0]].append(j)
        else:
            lines[i].append(j)

        if j not in follows:
            follows[j] = [i]
        else:
            follows[j].append(i)
    return lines, follows

In [3]:
def final_file_line_by_line(file_as_dict, threshold):
    final_file_as_dict = {'full_line':[], 'page_id':[], 'ymax_max':[], 'ymin_min':[], 'bboxes': []}
    lines_to_merge = get_lines_to_merge(file_as_dict, threshold)
    line_merge_map, follow_merge_map = line_merging(lines_to_merge)

    # final_file_as_dict['bboxes'] = [bbox for bbox in file_as_dict['bboxes']]
    for i in range(len(file_as_dict['words'])):
        if i not in line_merge_map and i not in follow_merge_map:
            final_file_as_dict['full_line'].extend([file_as_dict['words'][i]])
            final_file_as_dict['bboxes'].extend([file_as_dict['bboxes'][i]])
            final_file_as_dict['page_id'].append(file_as_dict['page_id'][i])
            final_file_as_dict['ymin_min'].append(file_as_dict['ymin_ymax'][i][0])
            final_file_as_dict['ymax_max'].append(file_as_dict['ymin_ymax'][i][1])
            
        else:
            if i in line_merge_map:
                line = []
                ymin_min = []
                ymax_max = []
                bboxes = []
                for j in line_merge_map[i]:
                    line.extend(file_as_dict['words'][j])
                    ymin_min.append(file_as_dict['ymin_ymax'][j][0])
                    ymax_max.append(file_as_dict['ymin_ymax'][j][1])
                    bboxes.extend(file_as_dict['bboxes'][j])
                
                final_file_as_dict['full_line'].append(line)
                final_file_as_dict['bboxes'].append(bboxes)
                final_file_as_dict['ymin_min'].append(ymin_min)
                final_file_as_dict['ymax_max'].append(ymax_max)
                final_file_as_dict['page_id'].append(file_as_dict['page_id'][i])
                
                #final_file_as_dict['full_line'].append([file_as_dict['words'][j] for j in line_merge_map[i]])
                #final_file_as_dict['ymin_min'].append([file_as_dict['ymin_ymax'][j][0] for j in line_merge_map[i]])
                #final_file_as_dict['ymax_max'].append([file_as_dict['ymin_ymax'][j][1] for j in line_merge_map[i]])
    
    
    for idx, (min_element, max_element) in enumerate(zip(final_file_as_dict['ymin_min'], final_file_as_dict['ymax_max'])):
        if type(min_element) == type(list()):
            new_min_element = min(min_element)
            new_max_element = max(max_element)
            final_file_as_dict['ymin_min'][idx] = new_min_element
            final_file_as_dict['ymax_max'][idx] = new_max_element
    
    print(len(final_file_as_dict['full_line']), len(final_file_as_dict['bboxes']))
    return final_file_as_dict

def get_toc_page(preprocessed_output):
    regex_exp = r"(table of contents|tableof(?:contents)?|(?:table\s)?of*conten|contents?)"
    
    for page_id, line in zip(preprocessed_output['page_id'],
                             preprocessed_output['full_line']):
        if re.search(regex_exp, " ".join(line).lower()):
            return page_id
    return None

def match_line(section_info, preprocessed_output, idx1, idx2,
               subset_match_threshold, line_len_match_threshold,
               beg_line_match_threshold, first_line_match_threshold, toc_page):
    '''
    replace bulk of find_start logic with this. this function operates 1 line at a time or 2 lines at a time
    if both idx are passed in. -- need to figure out how to incorporate proper bboxes logic though
    '''
    
    if preprocessed_output['page_id'][idx1] == toc_page:
        return None, None
    
    if (idx1 is not None) and (idx2 is not None):
        if preprocessed_output['page_id'][idx2] == toc_page:
            return None, None
        if preprocessed_output['page_id'][idx1] != preprocessed_output['page_id'][idx2]:
            return None, None
        
        ## 2 line merge
        multi_line = " ".join(preprocessed_output['full_line'][idx1] + preprocessed_output['full_line'][idx2])
        line1 = " ".join(preprocessed_output['full_line'][idx1])
        
        beg_line = multi_line[0: len(section_info[0])*2] # not really necessary for this case but keeping it for consistency
        
        if (fuzz.partial_ratio(section_info[0].lower(), multi_line.lower()) > subset_match_threshold       and
            len(multi_line) >= len(section_info[0]) * line_len_match_threshold                             and
            fuzz.partial_ratio(section_info[0].lower(), beg_line.lower()) > beg_line_match_threshold and
            fuzz.partial_ratio(section_info[0].lower(), line1.lower()) > first_line_match_threshold):

            # check if the beginning of the section info (first 2 words- design choice )has a high match with 
            # the beginning of the matched line. 
            section_info_beg = section_info[0].split()[:2]
            section_info_beg = " ".join(section_info_beg)

            matched_line_beg = multi_line.split()[:2]
            matched_line_beg = " ".join(matched_line_beg)

            # if debug:
                # print(f"section_info_beg: {section_info_beg}\nmatched_line_beg: {matched_line_beg}, match: {fuzz.partial_ratio(section_info_beg.lower(), matched_line_beg.lower())}")

            if fuzz.partial_ratio(section_info_beg.lower(), matched_line_beg.lower()) > subset_match_threshold:
                ymin = preprocessed_output['ymin_min'][idx1] # min of first line
                ymax = preprocessed_output['ymax_max'][idx2] # max of second line
                
                page_id = preprocessed_output['page_id'][idx1]
                bboxes = [preprocessed_output['bboxes'][idx1], preprocessed_output['bboxes'][idx2]]
                line2 = " ".join(preprocessed_output['full_line'][idx2])
                
                return (line1, line2, section_info[0], ymin, ymax, page_id, bboxes), idx2 + 1
            
            # if the beginning of the section info and the matched line don't have a high match then
            # return None
            else:
                return None, None
        
        return None, None

    line = " ".join(preprocessed_output['full_line'][idx1])
    beg_line = line[0: len(section_info[0])*2]

    if (fuzz.partial_ratio(section_info[0].lower(), line.lower()) > subset_match_threshold and
        len(line) >= len(section_info[0]) * line_len_match_threshold                       and
        fuzz.partial_ratio(section_info[0].lower(), beg_line.lower()) > beg_line_match_threshold):
        
        section_info_beg = section_info[0].split()[:2]
        section_info_beg = " ".join(section_info_beg)

        matched_line_beg = line.split()[:2]
        matched_line_beg = " ".join(matched_line_beg)

        # if debug:
        # print(f"section_info_beg: {section_info_beg}\nmatched_line_beg: {matched_line_beg}, match: {fuzz.partial_ratio(section_info_beg.lower(), matched_line_beg.lower())}")

        if fuzz.partial_ratio(section_info_beg.lower(), matched_line_beg.lower()) > subset_match_threshold:
            ymin = preprocessed_output['ymin_min'][idx1]
            ymax = preprocessed_output['ymax_max'][idx1]
            page_id = preprocessed_output['page_id'][idx1]
            bboxes = preprocessed_output['bboxes'][idx1]

            return (line, None, section_info[0], ymin, ymax, page_id, bboxes), idx1 + 1
        
        else:
            None, None
    
    return None, None
    
def find_start_new(section_info, preprocessed_output,
                   subset_match_threshold, line_len_match_threshold,
                   beg_line_match_threshold, first_line_match_threshold, 
                   last_line_pointer):
    '''
    Given a toc section title, iterate all the lines in the file from the last line associated with a section title
    going forward
    '''
    
    if last_line_pointer == len(preprocessed_output['full_line']):
        return None, last_line_pointer
    
    toc_page = get_toc_page(preprocessed_output)
    
    itertuple = zip(range(last_line_pointer, len(preprocessed_output['full_line'])),
                    range(last_line_pointer + 1, len(preprocessed_output['full_line'])))
    
    for idx1, idx2 in itertuple:
        
        ## first try matching with first line
        match = match_line(section_info, preprocessed_output, idx1, None,
                           subset_match_threshold, line_len_match_threshold,
                           beg_line_match_threshold, first_line_match_threshold, 
                           toc_page)
        if match[0]:
            return match
        
        ## let's try matching with second line only. This code is extremely inefficient but keeping for now.
        match = match_line(section_info, preprocessed_output, idx2, None,
                           subset_match_threshold, line_len_match_threshold,
                           beg_line_match_threshold, first_line_match_threshold, 
                           toc_page)
        if match[0]:
            return match
    

        # now try matching with 2 lines 
        match = match_line(section_info, preprocessed_output, idx1, idx2,
                           subset_match_threshold, line_len_match_threshold,
                           beg_line_match_threshold, first_line_match_threshold, 
                           toc_page)
        if match[0]:
            return match
        ## if no match, move onto the next pair of lines
    
    return None, last_line_pointer

def get_starts_all(section_dict, preprocessed_output):
    last_line_pointer = 0
    #lines = list(zip(preprocessed_output2['page_id'],
    #                  preprocessed_output2['full_line'],
    #                  preprocessed_output2['ymin_min'],
    #                  preprocessed_output2['ymax_max']))


    subset_match_threshold = 80    # Design decision to only allow subset match ratios of > 80/100
    line_len_match_threshold = 0.8 # Design decision to potentially only match document lines
                                   # that are not much smaller than table of content label
    beg_line_match_threshold = 80
    first_line_match_threshold = 20 # in case of merged lines lets make sure top line is at least mildly relevant
    starts = []

    for key, section_info in section_dict.items():

        start, last_line_pointer = find_start_new(section_info, preprocessed_output,
                                                  subset_match_threshold, line_len_match_threshold,
                                                  beg_line_match_threshold, first_line_match_threshold, 
                                                  last_line_pointer)
        if start:
            starts.append(start)
        else:
            print(f"Couldn't match {section_info[0]} with a line. Moving onto next TOC section")
    
    return starts

def flatten_contract_dict(nested_dict):
    i = 1
    section_dict_flattened = {}

    for item in nested_dict.items():
        section_dict_flattened[i] = (item[1][0], {})
        i += 1
        for sub_item in item[1][1].items():
            section_dict_flattened[i] = (sub_item[1][0], {})
            i += 1
    return section_dict_flattened

In [4]:
def merge_bboxes(line1_words, line2_words, query, bboxes, window_size, match_score):
    if isinstance(line1_words, str):
        line1_words = line1_words.split()

    if isinstance(line2_words, str):
        line2_words = line2_words.split()

    if line2_words:
        full_line_words = line1_words + line2_words
    else:
        full_line_words = line1_words
    
    # print(line1_words)
    # print(line2_words)
    # print(full_line_words)
    # print(bboxes)
    
    max_window_score, match_idx = 0, 0
    for idx, start_idx in enumerate(range(0, len(full_line_words), window_size)):
        window_text = " ".join(full_line_words[start_idx : start_idx + window_size])
        
        window_score = process.extractBests(window_text, [query], scorer=fuzz.token_set_ratio)[0][-1]

        if line2_words:
            candidate_bboxes = bboxes[0] + bboxes[1]
        else:
            candidate_bboxes = bboxes[start_idx : start_idx + window_size]

        if window_score >= match_score:
            # x min is the x_left of the first bbox 
            # y_min is the min of the top left y's for each box
            # x_min = candidate_bboxes[0][0][0]
            x_min = min([x[0][0] for x in candidate_bboxes])
            y_min = min([y[0][-1] for y in candidate_bboxes])

            # x max is the x_right of the last bbox 
            # y_max is the max of the bottom_right y's for each box
            # x_max = candidate_bboxes[-1][1][0]
            x_max = max([x[1][0] for x in candidate_bboxes])
            y_max = max([y[1][-1] for y in candidate_bboxes])

            merged_bbox = [ [x_min, y_min], [x_max, y_max] ]
            
            return merged_bbox, window_text
        
        else:
            if window_score > max_window_score:
                max_window_score = window_score
                match_text = window_text
                match_candidate_bboxes = candidate_bboxes

            continue
    
    # in case the window_text has a lower match score than with the entire string, match with the 
    # segment with highest matching score
    # x_min = match_candidate_bboxes[0][0][0]
    x_min = min([x[0][0] for x in match_candidate_bboxes])
    y_min = min([y[0][-1] for y in match_candidate_bboxes])

    # x max is the x_right of the last bbox 
    # y_max is the max of the bottom_right y's for each box
    # x_max = match_candidate_bboxes[-1][1][0]
    x_max = max([x[1][0] for x in match_candidate_bboxes])
    y_max = max([y[1][-1] for y in match_candidate_bboxes])

    merged_bbox = [ [x_min, y_min], [x_max, y_max] ]

    return merged_bbox, match_text

def extract_exact_match(row):
    
    query = row['Section Title via HTML']
    if query is not None:
        line1 = row.loc['Line via OCR']
        line2 = row.loc['Line2 via OCR']

        if line2 is not None:
            full_line = line1 + line2
        else:
            full_line = line1
        
        match = process.extractBests(full_line, [query], scorer=fuzz.token_set_ratio)
        
        match_text, match_score = match[0]
        window_size = len(match_text.split())
        
        
        merged_bbox, exact_match_text = merge_bboxes(line1, 
                                                     line2,
                                                     query, 
                                                     row['bboxes'], 
                                                     window_size, 
                                                     match_score)
        
        return merged_bbox, exact_match_text
    
    else:
        return None, None

In [5]:
def make_images(poppler_path, parent_path, pdf_path, contract_name_mapper_reverse):
    pdf_name = pdf_path.split("/")[-1][:-4]
    print(f"Converting pdf to image for {pdf_name}")
    #pdf_name = pdf_name if len(pdf_name) < 20 else pdf_name[:20]
    contract_name = contract_name_mapper_reverse[pdf_name]
    full_output_folder = parent_path + contract_name
    if not os.path.exists(full_output_folder):
        os.mkdir(full_output_folder)
        print(f"Saving images in {full_output_folder}")
    convert_from_path(pdf_path=pdf_path, output_folder=full_output_folder, poppler_path=poppler_path, fmt='jpeg')
    img_files = os.listdir(full_output_folder)
    
    page_tracking = {
        'dimensions':None, 
        'bbox':None,
        'path_to_image':None,
        'path_to_image_with_bounding_box':None,
    }
    pages_list = []
    for img_name in img_files:
        pages_list.append(page_tracking.copy())
        
        img_read_path = full_output_folder + "/" + img_name
        pages_list[-1]['path_to_image'] = img_read_path
        img = cv2.imread(img_read_path)
        pages_list[-1]['dimensions'] = PIL.Image.open(img_read_path).size
        
    contract_tracker = {
        contract_name:{'pages':pages_list,
                      'path_to_pdf':None,
                      'path_to_html':None,},
    }
    
    return full_output_folder, contract_tracker, contract_name

def write_bbox_images(pdf_parsed_df, full_output_folder, contract_tracker, contract_name):
    '''
    Returns json outpath file with info
    '''
    for page_id in range(len(contract_tracker[contract_name]['pages'])):
        if page_id not in pdf_parsed_df['page_id'].to_list():
            continue
        sub_df = df[df['page_id'] == page_id].reset_index(drop=True).copy()

        img_page_id = f"{page_id + 1}.jpg"
        img_name = list(filter(lambda x: True if img_page_id in x else False, os.listdir(full_output_folder)))[0]
        img_read_path = full_output_folder + "/" + img_name
        img_write_parent_path = f"{full_output_folder}/bboxes/"

        if not os.path.exists(img_write_parent_path):
            os.mkdir(img_write_parent_path)

        img_write_full_path = img_write_parent_path + img_name
        img = cv2.imread(img_read_path)
        width, height = PIL.Image.open(img_read_path).size
        
        bbox_attributes = {'corr':None, 'text_via_ocr':None, 'text_from_html':None}
        contract_tracker[contract_name]['pages'][page_id]['bbox'] = [bbox_attributes for _ in range(len(sub_df))]
        contract_tracker[contract_name]['pages'][page_id]['path_to_image_with_bounding_box'] = img_write_full_path
                                                      
        for idx, row in sub_df.iterrows():
            (x1, y1), (x2, y2) = row['exact_match_bbox']

            x1 = math.floor(width * x1)
            x2 = math.ceil(width * x2)
            y1 = math.floor(height * y1)
            y2 = math.ceil(height * y2)

            img = cv2.rectangle(img, (x1, y1), (x2, y2), (255,0,0), 2)
            print(f"Drew bounding boxes for {img_name} page ")
            
            contract_tracker[contract_name]['pages'][page_id]['bbox'][idx]['corr'] = ((x1, y1), (x2, y2))
            contract_tracker[contract_name]['pages'][page_id]['bbox'][idx]['text_via_ocr'] = row['exact_match_text']
            contract_tracker[contract_name]['pages'][page_id]['bbox'][idx]['text_from_html'] = row['Section Title via HTML']
            
            
        cv2.imwrite(img_write_full_path, img)
        print(f"Wrote image with bboxes @ {img_write_full_path}")
    return contract_tracker

In [6]:
def run_matching(contract_key, section_dicts, path_dict):
    section_dict = section_dicts[path_dict["section_dict_key"]]
    section_dict = flatten_contract_dict(section_dict)
    doctr_output_path = path_dict["doctr_output_json"]
    
    with open(doctr_output_path, 'r') as f:
        doctr_output = json.load(f)
        
    file_as_dict = get_file_by_dict(doctr_output)
    preprocessed_output = final_file_line_by_line(file_as_dict, threshold=0.65)
    df = pd.DataFrame(preprocessed_output)
    preprocessed_output = df.sort_values(by=['page_id', 'ymin_min']).to_dict(orient='list')

    df = pd.DataFrame(get_starts_all(section_dict, preprocessed_output)).rename(columns={0:'Line via OCR',
                                                                                         1:'Line2 via OCR',
                                         2:'Section Title via HTML',
                                         3:'ymin',
                                         4:'ymax',
                                         5:'page_id',
                                         6:'bboxes'})
    df['exact_match_bbox'], df['exact_match_text'] = zip(*df.apply(lambda row: extract_exact_match(row), axis=1))
    return df

In [7]:
sample_files = ['CUAD_v1/full_contract_pdf/Part_I/Development/HarpoonTherapeuticsInc_20200312_10-K_EX-10.18_12051356_EX-10.18_Development Agreement_Option Agreement.pdf',
                'CUAD_v1/full_contract_pdf/Part_I/Distributor/ZogenixInc_20190509_10-Q_EX-10.2_11663313_EX-10.2_Distributor Agreement.pdf',
                'CUAD_v1/full_contract_pdf/Part_I/Endorsement/PerformanceSportsBrandsInc_20110909_S-1_EX-10.10_7220214_EX-10.10_Endorsement Agreement.pdf',
                'CUAD_v1/full_contract_pdf/Part_III/Marketing/NUVEEN - REMARKETING AGREEMENT.PDF',
                'CUAD_v1/full_contract_pdf/Part_III/IP/OTISWORLDWIDECORP_04_03_2020-EX-10.4-INTELLECTUAL PROPERTY AGREEMENT by and among UNITED TECHNOLOGIES CORPORATION, OTIS WORLDWIDE CORPORATION and CARRIER ~1.PDF',
                'CUAD_v1/full_contract_pdf/Part_I/Outsourcing/ParatekPharmaceuticalsInc_20170505_10-KA_EX-10.29_10323872_EX-10.29_Outsourcing Agreement.pdf']

In [8]:
## enter basic local path info here
poppler_path = "C:/Users/islam/poppler-22.04.0/Library/bin/"
all_section_dicts_path = "toc_extracted.json"
all_doctr_json_outputs_path = "json_outputs/"
cuad_master_parent_path = "C:/Users/islam/Desktop/shortcutpaths/CUAD_v1/"  ## this will be joined to relative pdf path
image_output_parent_path = "cuad_images_labeled/"

master_path_dictionary = dict()

with open(all_section_dicts_path, "r") as f:
    section_dicts = json.load(f)

for idx, (section_dict_key, section_dict) in enumerate(section_dicts.items()):
    key = f"contract_{idx}"
    master_path_dictionary[key] = dict()
    master_path_dictionary[key]['section_dict_key'] = section_dict_key
    master_path_dictionary[key]['doctr_output_json'] = all_doctr_json_outputs_path + section_dict_key[:-4] + "json"
    
    for rel_pdf_path in sample_files:
        if section_dict_key[:-4] in rel_pdf_path:
            master_path_dictionary[key]['pdf_path'] = cuad_master_parent_path + rel_pdf_path
    
    master_path_dictionary[key]['text_file_path'] = cuad_master_parent_path + "CUAD_v1/full_contract_txt/" + section_dict_key[:-4] + "txt"

In [9]:
master_path_dictionary

{'contract_0': {'section_dict_key': 'ZogenixInc_20190509_10-Q_EX-10.2_11663313_EX-10.2_Distributor Agreement.html',
  'doctr_output_json': 'json_outputs/ZogenixInc_20190509_10-Q_EX-10.2_11663313_EX-10.2_Distributor Agreement.json',
  'pdf_path': 'C:/Users/islam/Desktop/shortcutpaths/CUAD_v1/CUAD_v1/full_contract_pdf/Part_I/Distributor/ZogenixInc_20190509_10-Q_EX-10.2_11663313_EX-10.2_Distributor Agreement.pdf',
  'text_file_path': 'C:/Users/islam/Desktop/shortcutpaths/CUAD_v1/CUAD_v1/full_contract_txt/ZogenixInc_20190509_10-Q_EX-10.2_11663313_EX-10.2_Distributor Agreement.txt'},
 'contract_1': {'section_dict_key': 'PerformanceSportsBrandsInc_20110909_S-1_EX-10.10_7220214_EX-10.10_Endorsement Agreement.html',
  'doctr_output_json': 'json_outputs/PerformanceSportsBrandsInc_20110909_S-1_EX-10.10_7220214_EX-10.10_Endorsement Agreement.json',
  'pdf_path': 'C:/Users/islam/Desktop/shortcutpaths/CUAD_v1/CUAD_v1/full_contract_pdf/Part_I/Endorsement/PerformanceSportsBrandsInc_20110909_S-1_EX-10

In [10]:
contract_info_mapper = json.load(open("contract_full_map.json"))
contract_name_mapper = json.load(open("contract_name_map.json"))
contract_name_mapper_reverse = {value:key for key,value in contract_name_mapper.items()}

In [11]:
output_jsons = []
for contract_key, path_dict in master_path_dictionary.items():

    
    print(f"On contract {contract_key}")
    
    try:
        df = run_matching(contract_key=contract_key,
                      section_dicts=section_dicts, 
                      path_dict=path_dict)

        (full_output_folder, 
         contract_tracker, 
         contract_name) = make_images(poppler_path=poppler_path, 
                                          parent_path=image_output_parent_path, 
                                          pdf_path=path_dict['pdf_path'],
                                          contract_name_mapper_reverse=contract_name_mapper_reverse)

        output_jsons.append(write_bbox_images(pdf_parsed_df=df,
                                full_output_folder=full_output_folder,
                                contract_tracker=contract_tracker,
                                contract_name=contract_name))
    except Exception as e:
        print(e)
        print("Moving onto next contract")

On contract contract_0
1858 1858
Couldn't match 1 0.1 Mutual Representations, Warranties and Covenants with a line. Moving onto next TOC section
Couldn't match 1 0.2 Representations, Warranties and Covenants of Distributor with a line. Moving onto next TOC section
Couldn't match 1 0.3 Representations and Warranties of Zogenix with a line. Moving onto next TOC section
Couldn't match 1 0.4 Disclaimer with a line. Moving onto next TOC section
Couldn't match 1 0.5 Limitation of Liability with a line. Moving onto next TOC section
Couldn't match 1 1.1 Indemnification of Zogenix with a line. Moving onto next TOC section
Couldn't match 1 1.2 Indemnification of Distributor with a line. Moving onto next TOC section
Couldn't match 1 1.3 Procedure with a line. Moving onto next TOC section
Couldn't match 1 1.4 Insurance with a line. Moving onto next TOC section
Couldn't match 1 2.1 Term with a line. Moving onto next TOC section
Couldn't match 1 2.2 Termination with a line. Moving onto next TOC sect

Wrote image with bboxes @ cuad_images_labeled/ct_506/bboxes/3eea96cf-28e5-455b-aae1-8c5399f42e33-39.jpg
Drew bounding boxes for 3eea96cf-28e5-455b-aae1-8c5399f42e33-40.jpg page 
Drew bounding boxes for 3eea96cf-28e5-455b-aae1-8c5399f42e33-40.jpg page 
Wrote image with bboxes @ cuad_images_labeled/ct_506/bboxes/3eea96cf-28e5-455b-aae1-8c5399f42e33-40.jpg
Drew bounding boxes for 3eea96cf-28e5-455b-aae1-8c5399f42e33-41.jpg page 
Drew bounding boxes for 3eea96cf-28e5-455b-aae1-8c5399f42e33-41.jpg page 
Drew bounding boxes for 3eea96cf-28e5-455b-aae1-8c5399f42e33-41.jpg page 
Wrote image with bboxes @ cuad_images_labeled/ct_506/bboxes/3eea96cf-28e5-455b-aae1-8c5399f42e33-41.jpg
Drew bounding boxes for 3eea96cf-28e5-455b-aae1-8c5399f42e33-56.jpg page 
Drew bounding boxes for 3eea96cf-28e5-455b-aae1-8c5399f42e33-56.jpg page 
Drew bounding boxes for 3eea96cf-28e5-455b-aae1-8c5399f42e33-56.jpg page 
Drew bounding boxes for 3eea96cf-28e5-455b-aae1-8c5399f42e33-56.jpg page 
Wrote image with bboxe

Drew bounding boxes for f804f44a-ddd9-4085-b252-7666d0ae4457-16.jpg page 
Drew bounding boxes for f804f44a-ddd9-4085-b252-7666d0ae4457-16.jpg page 
Drew bounding boxes for f804f44a-ddd9-4085-b252-7666d0ae4457-16.jpg page 
Drew bounding boxes for f804f44a-ddd9-4085-b252-7666d0ae4457-16.jpg page 
Wrote image with bboxes @ cuad_images_labeled/ct_308/bboxes/f804f44a-ddd9-4085-b252-7666d0ae4457-16.jpg
Drew bounding boxes for f804f44a-ddd9-4085-b252-7666d0ae4457-17.jpg page 
Drew bounding boxes for f804f44a-ddd9-4085-b252-7666d0ae4457-17.jpg page 
Drew bounding boxes for f804f44a-ddd9-4085-b252-7666d0ae4457-17.jpg page 
Drew bounding boxes for f804f44a-ddd9-4085-b252-7666d0ae4457-17.jpg page 
Drew bounding boxes for f804f44a-ddd9-4085-b252-7666d0ae4457-17.jpg page 
Drew bounding boxes for f804f44a-ddd9-4085-b252-7666d0ae4457-17.jpg page 
Wrote image with bboxes @ cuad_images_labeled/ct_308/bboxes/f804f44a-ddd9-4085-b252-7666d0ae4457-17.jpg
Drew bounding boxes for f804f44a-ddd9-4085-b252-7666

Couldn't match 5.5 ['Reference Standards', '18'] with a line. Moving onto next TOC section
Couldn't match 6.1 ['Placement of Purchase Orders', '19'] with a line. Moving onto next TOC section
Couldn't match 6.2 ['Acceptance of Orders', '19'] with a line. Moving onto next TOC section
Couldn't match 6.3 ['Delays', '19'] with a line. Moving onto next TOC section
Couldn't match 6.4 ['Cancellation of Purchase Orders', '20'] with a line. Moving onto next TOC section
Couldn't match 6.5 ['Material Failure of Supply', '20'] with a line. Moving onto next TOC section
Couldn't match 6.6 ['Services', '20'] with a line. Moving onto next TOC section
Couldn't match 7.1 ['Storage of Product', '20'] with a line. Moving onto next TOC section
Couldn't match 7.2 ['Release and Shipment of Product', '21'] with a line. Moving onto next TOC section
Couldn't match 7.3 ['Documentation', '21'] with a line. Moving onto next TOC section
Couldn't match 7.4 ['Steering Committee', '21'] with a line. Moving onto next TO

Wrote image with bboxes @ cuad_images_labeled/ct_345/bboxes/2f3c2ac6-b7d3-4ac9-af56-59b9fd05fc3b-22.jpg
Drew bounding boxes for 2f3c2ac6-b7d3-4ac9-af56-59b9fd05fc3b-23.jpg page 
Wrote image with bboxes @ cuad_images_labeled/ct_345/bboxes/2f3c2ac6-b7d3-4ac9-af56-59b9fd05fc3b-23.jpg
Drew bounding boxes for 2f3c2ac6-b7d3-4ac9-af56-59b9fd05fc3b-25.jpg page 
Wrote image with bboxes @ cuad_images_labeled/ct_345/bboxes/2f3c2ac6-b7d3-4ac9-af56-59b9fd05fc3b-25.jpg
Drew bounding boxes for 2f3c2ac6-b7d3-4ac9-af56-59b9fd05fc3b-26.jpg page 
Wrote image with bboxes @ cuad_images_labeled/ct_345/bboxes/2f3c2ac6-b7d3-4ac9-af56-59b9fd05fc3b-26.jpg
Drew bounding boxes for 2f3c2ac6-b7d3-4ac9-af56-59b9fd05fc3b-27.jpg page 
Wrote image with bboxes @ cuad_images_labeled/ct_345/bboxes/2f3c2ac6-b7d3-4ac9-af56-59b9fd05fc3b-27.jpg
Drew bounding boxes for 2f3c2ac6-b7d3-4ac9-af56-59b9fd05fc3b-29.jpg page 
Wrote image with bboxes @ cuad_images_labeled/ct_345/bboxes/2f3c2ac6-b7d3-4ac9-af56-59b9fd05fc3b-29.jpg
Drew b

In [87]:
output_jsons

[{'ct_506': {'pages': [{'dimensions': (1653, 2339),
     'bbox': None,
     'path_to_image': 'cuad_images_labeled/ct_506/69a9eabe-3c93-4340-839a-3ff3566ed205-01.jpg',
     'path_to_image_with_bounding_box': None},
    {'dimensions': (1653, 2339),
     'bbox': None,
     'path_to_image': 'cuad_images_labeled/ct_506/69a9eabe-3c93-4340-839a-3ff3566ed205-02.jpg',
     'path_to_image_with_bounding_box': None},
    {'dimensions': (1653, 2339),
     'bbox': None,
     'path_to_image': 'cuad_images_labeled/ct_506/69a9eabe-3c93-4340-839a-3ff3566ed205-03.jpg',
     'path_to_image_with_bounding_box': None},
    {'dimensions': (1653, 2339),
     'bbox': None,
     'path_to_image': 'cuad_images_labeled/ct_506/69a9eabe-3c93-4340-839a-3ff3566ed205-04.jpg',
     'path_to_image_with_bounding_box': None},
    {'dimensions': (1653, 2339),
     'bbox': None,
     'path_to_image': 'cuad_images_labeled/ct_506/69a9eabe-3c93-4340-839a-3ff3566ed205-05.jpg',
     'path_to_image_with_bounding_box': None},
    {'